# Detecting insults in Social Media

At this competition, first "train.csv" was made avialable and contestant were ask to submit their preliminary models. Then public and private leaderboad evaluation at this stage was done on a test data set that later was made avialable as "test_with_solutions.csv". The contastents then submitted their final models which were evaluated on a verfication set. This set was provided after the competition was disclosed as "impermium_verification_labels.csv".



In [1]:
import pandas as pd
train = pd.read_csv("data/train.csv")
train.sample()

,Insult,Date,Comment
0,1,20120618192155Z,"""You fuck your dad."""
1,0,20120528192215Z,"""i really don't understand your point.\xa0 It ..."
2,0,NaN,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,0,NaN,"""listen if you dont wanna get married to a man..."
4,0,20120619094753Z,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [2]:
test_with_solutions = pd.read_csv("data/test_with_solutions.csv")
verfication_set = pd.read_csv("data/impermium_verification_labels.csv")

In [3]:
train_total = pd.concat([train, test_with_solutions.iloc[:, :-1]]).reset_index()

## Text Cleaning

In [5]:
train_total_clean = train_total.copy()
train_total_clean.Comment = comments

In [1]:
from scipy import sparse
from sklearn.base import BaseEstimator, TransformerMixin

class Preprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        comments_clean = []
        for c in X:
            c = c.replace('\\\\', '\\')
            c = c.replace('\\n', ' ')
            #c = c.replace('\'', '')
            c = re.sub(r'http[s]?://[^\s]*', ' ', c)
            c = re.sub(r'\\x[0-9a-f]{2}', ' ', c)
            c = re.sub(r'\\u[0-9a-f]{4}', ' ', c)
            c = re.sub(r'[-"]', '', c)
            c = re.sub(r'[:*#%&,.?!\']', ' ', c)
            c = re.sub(r"(.)\1{2,}", '\g<1>', c)
            #c = re.sub(r'@[^ ]*', ' ', c)
            #c = re.sub(r'[0-9]+', ' ', c)
            c = " ".join([wordnet_lemmatizer.lemmatize(wordnet_lemmatizer.lemmatize(w, pos='v')).lower() 
                          for w in c.split()])

            #c = " ".join([w.lower() for w in c.split()])# if 3<=len(w)<=40])
            comments_clean.append(c)
        return comments_clean

In [31]:
prep = Preprocessor()

In [32]:
prep.fit_transform(comments)

['you fuck your dad',
 u'i really don t understand your point it seem that you be mix apple and orange',
 u'a majority of canadians can and have be wrong before now and will be again unless you re supportive of the idea that nothing be full proof or perfect so you take your chance and if we should inadvertently kill your son or daughter then them s the break and we can always regard you a collateral damage like in wartime and sorry but the cheque in the mail',
 u'listen if you dont wanna get marry to a man or a woman dont do it what would it bother you if gay people get marry stay in your lane do you let them do them and your god be so nice but quick to judg if your not like him think you wasnt suppose to judge people',
 'c c b n xu ng ng bi u t nh 2011 c n ho kh ng c c ng d n ng i cu u chi nh c c n ho kh ng c c n ng d n gi t v n giang c n th c n ho kh ng r t cu c c g th ch ng ta bi t ai c ng y u chu ng ho b nh nh ng i khi ho b nh ch th t s n sau chi n tranh m th i kh ng c n con ng n o

In [48]:

from nltk import word_tokenize
import nltk
l = nltk.pos_tag(word_tokenize(comments[1]))
from scipy import sparse

from sklearn.base import BaseEstimator
class FeatureExtractor(BaseEstimator):
    def __init__(self):
        with open('bad_words.txt') as f:
            self.badwords = set(w.strip() for w in f.readlines())
        
    def fit(self, X, y=None):
        self.fit_transform(X, y)
        return self

    def transform(self, X):
        return self.fit_transform(X)

    def fit_transform(self, X, y=None):

        pos_tags = ["VB", "JJ", "RB", "NN"]
        def pos_tag_count(comment):
            pos = nltk.pos_tag(word_tokenize(comment))
            d = {}
            for tag in pos_tags:
                d[tag] = 0
            for p in pos:
                for tag in pos_tags:
                    if p[1].startswith(tag):
                        d[tag] += 1
            return d.values()
    
        def bad_word_count(comment):
            #comment = " ".join([wordnet_lemmatizer.lemmatize(wordnet_lemmatizer.lemmatize(w, pos='v')) 
            #              for w in comment.split()])
            cnt = 0
            for w in comment.split():
                if w.strip() in self.badwords:
                    cnt += 1
            return cnt
        
        at_mention_cnt = len(re.findall(r'@[^\s]+', s))
        
        comments_features = []
        for c in X:
            features = pos_tag_count(c)
            features.append(bad_word_count(c))
            features.append(at_mention_cnt)
            comments_features.append(features)
        
        return sparse.csr_matrix(comments_features)
        
            

In [96]:
TfidfVectorizer?

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion


tfidv_char = TfidfVectorizer(ngram_range=(1, 5), analyzer='char', stop_words='english')
tfidv_word = TfidfVectorizer(ngram_range=(1, 3), analyzer='word', stop_words='english', min_df=3)
tfidv_word = None
fe = FeatureExtractor()

fu = FeatureUnion([('fe', fe), ('tfidv_char', tfidv_char), ('tfidv_word', tfidv_word)])
#clf_rf = RandomForestClassifier()
clf_rf = LogisticRegression(C=20)

estimators = [('prep', prep), ('fu', fu), ('clf', clf_rf)]
pl = Pipeline(estimators)

In [95]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

def scoring(estimator, X, y):
    y_pred = estimator.predict_proba(X)
    return roc_auc_score(y, y_pred[:, 1])

cross_val_score(pl, train_total.Comment, train_total.Insult, scoring=scoring)

array([0.90342138, 0.9048609 , 0.91351694])

In [51]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

def scoring(estimator, X, y):
    y_pred = estimator.predict_proba(X)
    return roc_auc_score(y, y_pred[:, 1])

cross_val_score(pl, train_total.Comment, train_total.Insult, scoring=scoring)

array([0.90349053, 0.9048758 , 0.91348496])

In [78]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

def scoring(estimator, X, y):
    y_pred = estimator.predict_proba(X)
    return roc_auc_score(y, y_pred[:, 1])

cross_val_score(pl, train_total.Comment, train_total.Insult, scoring=scoring)

array([0.89829617, 0.89590378, 0.90202589])

In [97]:
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'clf__C':np.logspace(-1, 2, 5),
              'fu__tfidv_char__ngram_range': [(1, 3), (1, 5), (1, 7)],
              'fu__tfidv_char__norm': ['l1', 'l2'],
              'fu__tfidv_char__min_df': [1, 2, 4],
              'fu__tfidv_char__stop_words': ['english', None],
             }
gs = GridSearchCV(pl, param_grid, scoring=scoring, n_jobs=4)
gs.fit(train_total.Comment, train_total.Insult)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('prep', Preprocessor()), ('fu', FeatureUnion(n_jobs=1,
       transformer_list=[('fe', FeatureExtractor()), ('tfidv_char', TfidfVectorizer(analyzer='char', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, m...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'fu__tfidv_char__min_df': [1, 2, 4], 'fu__tfidv_char__ngram_range': [(1, 3), (1, 5), (1, 7)], 'clf__C': array([  0.1    ,   0.56234,   3.16228,  17.78279, 100.     ]), 'fu__tfidv_char__norm': ['l1', 'l2'], 'fu__tfidv_char__stop_words': ['english', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=<function scoring at 0x7f8f2e391320>, verbose=0)

In [87]:
gs.best_params_

{'clf__C': 0.1, 'fu__tfidv_char__ngram_range': (1, 2)}

In [98]:
gs.best_params_

{'clf__C': 17.78279410038923,
 'fu__tfidv_char__min_df': 1,
 'fu__tfidv_char__ngram_range': (1, 7),
 'fu__tfidv_char__norm': 'l2',
 'fu__tfidv_char__stop_words': 'english'}

In [99]:
gs.best_score_

0.9096002277058917

In [100]:
clf = gs.best_estimator_

In [114]:
x  = clf.steps[1][1]

In [115]:
x.transformer_list

[('fe', FeatureExtractor()),
 ('tfidv_char',
  TfidfVectorizer(analyzer='char', binary=False, decode_error=u'strict',
          dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 7), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words='english', strip_accents=None, sublinear_tf=False,
          token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('tfidv_word', None)]

In [102]:
gs.best_score_

0.9096002277058917

In [57]:
cross_val_score(clf, train_total.Comment, train_total.Insult, scoring=scoring)

array([0.90370966, 0.90502588, 0.91382083])

In [59]:
cross_val_score(clf, verfication_set.Comment, verfication_set.Insult, scoring=scoring)

array([0.8213229 , 0.81389727, 0.80186759])

In [116]:

from sklearn.metrics import roc_auc_score
y_ver_pred = clf.predict_proba(verfication_set.Comment)[:, 1]
y_ver = verfication_set.Insult

roc_auc_score(y_ver, y_ver_pred)



0.8135985907248915

In [74]:
0.8177848818842078/0.84248

0.9706875912593864

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'clf__C':np.logspace(-1, 2, 5)}
gs = GridSearchCV(pl, param_grid, scoring=scoring, n_jobs=4)
gs.fit(train_total.Comment, train_total.Insult)